In [2]:
# from hgraph import *

import rdkit
import os
# import ipywidgets as widgets
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import py3Dmol
from IPython.display import display,HTML
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import time
from rdkit.Chem import Descriptors
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import rdDetermineBonds



def draw_with_spheres(mol):
    v = py3Dmol.view(width=300,height=300)
    mol_block = Chem.MolToMolBlock(mol)
    v.addModel(mol_block, "mol")
    v.zoomTo()
    v.setStyle({'sphere':{'radius':0.3},'stick':{'radius':0.2}})
    return v
    

In [3]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True

def draw_with_spheres_highlight_atoms(mol, index_map,highlight_atoms,n):
    v = py3Dmol.view(width=300,height=300)
    IPythonConsole.addMolToView(mol,v)
    v.zoomTo()
    v.setStyle({}, {'stick': {'radius': 0.2}, 'sphere': {'scale': 0.25}})

    ligand_highlightatoms=[]
    for i,atom in enumerate(mol.GetAtoms()):
        if(index_map[n][i] in highlight_atoms):
            ligand_highlightatoms.append(atom.GetIdx())


    # v.setStyle({'atom':highlight_atoms}, {'sphere': {'color': 'green', 'scale': 0.4}})

    v.setStyle({'serial':ligand_highlightatoms}, {'stick': {'radius': 0.2}, 'sphere': {'color': 'green', 'scale': 0.25}})

    conf = mol.GetConformer()
    for atom in mol.GetAtoms():
        if atom.GetIdx() in highlight_atoms:
            pos = conf.GetAtomPosition(atom.GetIdx())
            v.addLabel(atom.GetSymbol(), {'position': {'x': pos.x, 'y': pos.y, 'z': pos.z}, 'inFront' : True})
    return v

In [4]:
from hgraph.chemutils import get_smiles

METAL CHELATE

In [5]:
def read_molecule_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    atoms = []
    for line in lines:
        parts = line.split()
        if len(parts) == 4:
            atom_symbol, x, y, z = parts
            atoms.append((atom_symbol, float(x), float(y), float(z)))
    return atoms

In [6]:
file=["ZACODUR.xyz","ZFURROZ.xyz","ZFRMTFE.xyz","ZLECLAF.xyz","ZREHZEK.xyz"]
ligands_map={}

for f in file:
    rawmol=Chem.MolFromXYZFile(f)
    rdDetermineBonds.DetermineConnectivity(rawmol)
    rawmol.UpdatePropertyCache(strict=False)
    fe_atom = [atom for atom in rawmol.GetAtoms() if atom.GetSymbol() == 'Fe'][0]
    donor_atoms = [atom for atom in rawmol.GetAtoms() if rawmol.GetBondBetweenAtoms(fe_atom.GetIdx(), atom.GetIdx())]
    highlight_atoms=[]
    for atom in donor_atoms:
        highlight_atoms.append(atom.GetIdx())
    
    def get_ligand(mol, donor_atom, visited):
        ligand = []
        queue = list(donor_atom.GetNeighbors())
        
        while queue:
            current_atom = queue.pop(0)
            if current_atom.GetSymbol() != 'Fe' and current_atom.GetIdx() not in visited:
                visited.add(current_atom.GetIdx())
                ligand.append(current_atom)
                queue.extend(current_atom.GetNeighbors())
        return ligand

    ligands = []
    visited = set()

    for i, donor_atom in enumerate(donor_atoms):
    # print(donor_atom.GetSymbol(), donor_atom.GetIdx())
        if donor_atom.GetIdx() not in visited:
            ligand = get_ligand(rawmol, donor_atom, visited)
            ligands.append(ligand)

    atom_map={}

    for i,ligand in enumerate(ligands):
        # ligand_atoms={atom.GetSymbol() for atom in ligand}
        ligand_atoms={atom.GetIdx() : atom.GetSymbol() for atom in ligand}
        atom_map[i+1]=ligand_atoms
    
    mol_name=f.split('.')[0].strip()
    ligands_map[mol_name] = atom_map

    ligand_molblock=[]
    
    index_map={}
    for ligand_number, atom_dict in ligands_map[mol_name].items():
        atom_indices = list(atom_dict.keys())
        index_map[ligand_number]=atom_indices
        data=''
        data+=f"{len(atom_dict)}\n\n"
        metaldata=read_molecule_file(os.path.join(mol_name+'.xyz'))
        for atom_idx, atom_symbol in atom_dict.items():
            if(atom_idx < len(metaldata)):
                atom_symbol, x, y, z = metaldata[atom_idx]
                data+=f"{atom_symbol} {x} {y} {z}\n"
        ligand_molblock.append(data)
    
    print(f"Molecule: {mol_name}")
    
    # Now we have each individual ligand molblock and the corresponding indexes of the atoms is in index_map
    v = py3Dmol.view(linked=False, viewergrid=(1, len(ligand_molblock)+1), width=900, height=400)
    for i,ligandblock in enumerate(ligand_molblock):
        lmol=Chem.MolFromXYZBlock(ligandblock)
        rdDetermineBonds.DetermineConnectivity(lmol)
        lmol.UpdatePropertyCache(strict=False)
        print(f"ligand {i+1}")
        print(f"SMILES Before: {get_smiles(lmol)}")
        for j,atom in enumerate(lmol.GetAtoms()):
            # print(f"{atom.GetIdx()}:{atom.GetSymbol()}")
            if(index_map[i+1][j] in highlight_atoms):
                atom.SetAtomMapNum(2)
        
        if(i==0):
            mol_block = Chem.MolToMolBlock(rawmol)
            v.addModel(mol_block, "mol",viewer=(0,0))
            v.zoomTo(viewer=(0,0))
            v.setStyle({'sphere':{'radius':0.3},'stick':{'radius':0.2}},viewer=(0,0))
            label = True
            if label:
                conf = rawmol.GetConformer()
                for atom in rawmol.GetAtoms():
                    pos = conf.GetAtomPosition(atom.GetIdx())
                    v.addLabel(atom.GetSymbol(), {'position': {'x': pos.x, 'y': pos.y, 'z': pos.z}, 'inFront' : True},viewer=(0,0))
        
        mol_block = Chem.MolToMolBlock(lmol)  # Convert the RDKit molecule to a MolBlock
        v.addModel(mol_block, "mol", viewer=(0, i+1))  # Add the molecule to the specified viewer
        v.zoomTo(viewer=(0,i+1))
        v.setStyle({}, {'stick': {'radius': 0.2}, 'sphere': {'scale': 0.25}},viewer=(0,i+1))
        ligand_highlightatoms=[]
        for j,atom in enumerate(lmol.GetAtoms()):
            if(index_map[i+1][j] in highlight_atoms):
                ligand_highlightatoms.append(atom.GetIdx())
        v.setStyle({'serial':ligand_highlightatoms}, {'stick': {'radius': 0.2}, 'sphere': {'color': 'green', 'scale': 0.25}},viewer=(0,i+1))
        conf = lmol.GetConformer()
        for atom in lmol.GetAtoms():
            pos = conf.GetAtomPosition(atom.GetIdx())
            v.addLabel(atom.GetSymbol(), {'position': {'x': pos.x, 'y': pos.y, 'z': pos.z}, 'inFront' : True},viewer=(0,i+1))

        # view=draw_with_spheres_highlight_atoms(lmol,index_map,highlight_atoms,i+1)
        # v.addModel(Chem.MolToPDBBlock(rawmol), 'pdb')
        print(f"SMILES After: {get_smiles(lmol)}")
    
    v.show()
            



ArgumentError: Python argument types in
    rdkit.Chem.rdDetermineBonds.DetermineConnectivity(NoneType)
did not match C++ signature:
    DetermineConnectivity(RDKit::ROMol {lvalue} mol, bool useHueckel=False, int charge=0, double covFactor=1.3, bool useVdw=False)